# Requirements

In [1]:
import warnings
warnings.simplefilter('ignore')

import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.optimizers import Adam

Using TensorFlow backend.


# Model

## Generator

In [2]:
def get_generator(img_dim, sparse_code_dim):
    """
    Returns:
        keras.engine.sequential.Sequential: compiled genrator model
    """
    model = Sequential()
    model.add(Dense(512, input_shape=(img_dim,), activation="relu"))
    model.add(Dense(256, activation="relu"))
    model.add(Dense(128, activation="relu"))
    model.add(Dense(256, activation="relu"))
    model.add(Dense(512, activation="relu"))
    model.add(Dense(sparse_code_dim, activation="sigmoid"))
    model.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08), loss="binary_crossentropy")
    return model

## Discriminator

In [3]:
def get_discriminator(sparse_code_dim):
    """
    Returns:
        keras.engine.sequential.Sequential: compiled discriminator model
    """
    model = Sequential()
    model.add(Dense(512, input_shape=(sparse_code_dim,), activation="relu"))
    model.add(Dense(256, input_shape=(sparse_code_dim,), activation="relu"))
    model.add(Dense(128, input_shape=(sparse_code_dim,), activation="relu"))
    model.add(Dense(64, input_shape=(sparse_code_dim,), activation="relu"))
    model.add(Dense(32, input_shape=(sparse_code_dim,), activation="relu"))
    model.add(Dense(1, input_shape=(sparse_code_dim,), activation="sigmoid"))
    model.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08), loss="binary_crossentropy", metrics=["accuracy"])
    return model

## GAN

In [4]:
def make_gan(generator, discriminator):
    """
    Args:
        discriminator (keras.engine.sequential.Sequential): compiled discriminator model
        generator (keras.engine.sequential.Sequential): compilted generator model
    Returns:
        keras.engine.sequential.Sequential : compiled GAN model, 
    """
    model = Sequential()
    discriminator.trainable = False
    model.add(generator)
    model.add(discriminator)
    model.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08), loss="binary_crossentropy", metrics=["accuracy"])
    return model

# Training

In [5]:
def train(images, true_sparse_codes, epochs=200):
    """
    Args:
        epochs (int): amount of training iterations
    Returns:
        keras.engine.sequential.Sequential, keras.engine.sequential.Sequential: trained generator and discriminator
    
    """
    generator = get_generator()
    discriminator = get_discriminator()
    gan = make_gan(generator, discriminator)

    for i in range(epochs):

        # Get real sparse codes
        real_z = true_sparse_codes
        
        # Predict sparse codes with images
        false_z = generator.predict(images)

        # Construct batch
        batch = np.concatenate((real_z, false_z))

        # Initialize output
        y = np.concatenate((np.ones(batch_size), np.zeros(batch_size)))

        # Train discriminator
        accuracy = discriminator.train_on_batch(batch, y)

        # rain generator
        gan.train_on_batch(random_noise, np.ones(batch_size))

    return generator, discriminator

# Testing

In [ ]:
genrator, discriminator = train(images, true_sparse_codes, epochs=200)